In [6]:
from presidio_evaluator.data_generator import read_synth_dataset
from presidio_evaluator import ModelEvaluator
from collections import Counter

%load_ext autoreload
%autoreload 2

MY_PRESIDIO_ENDPOINT = "http://localhost:8080/api/v1/projects/test/analyze"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Evaluate your Presidio instance via the Presidio API

#### A. Read dataset for evaluation

In [9]:
input_samples = read_synth_dataset("../data/n2c2/set_2.json")
print("Read {} samples".format(len(input_samples)))

Read 237 samples


In [10]:
input_samples[0].full_text

"\n\n\nRecord date: 2092-11-16\n\n \n \n \n \n \n \nNovember 16, 2092\n \n \n \nStuart J. Long, M.D.\nGlenn Helen Medical Group\nGlenn Helen Infirmary\nAlgoma, CO  71183\n \nRE:  Roberta Vincent\n     MR #380-18-32-2\n \nDear Dr. Long:\n \nThank you for referring Ms. Roberta Vincent to the Renal Services Clinic.  I had the pleasure of seeing her on November 16,\n2092.  Please allow me to summarize her history for our records.\n \nMs. Vincent is a 56-year-old mother of eight with chronic renal\nfailure of unknown etiology.  She is six years status post\nhysterectomy for endometriosis during which an occult\nleiomyosarcoma was discovered.  She has an extensive history of\nhidradenitis, obesity, and hypertension.  She had one stillborn\ninfant, one premature infant, and three spontaneous abortions. \nSFOC records are unavailable at this time.\n \nHer medications include Premarin 0.625 qd, Diltiazem 240 qd,\nfurosemide 40 mg qd, Mevacor 20 mg qd, Calcitriol 0.25 mcg qd,\nNiferex 300 mg qd,

#### B. Descriptive statistics

In [11]:
flatten = lambda l: [item for sublist in l for item in sublist]

count_per_entity = Counter([span.entity_type for span in flatten([input_sample.spans for input_sample in input_samples])])
count_per_entity

Counter({'DATE_TIME': 1802,
         'PERSON': 1235,
         'ORGANIZATION': 404,
         'LOCATION': 365,
         'MEDICALRECORD': 168,
         'AGE': 339,
         'PROFESSION': 72,
         'IDNUM': 87,
         'USERNAME': 45,
         'PHONE_NUMBER': 48,
         'EMAIL_ADDRESS': 1})

#### C. Match the dataset's entity names

In [12]:
# Mapping between dataset entities and Presidio entities. Key: Dataset entity, Value: Presidio entity
entities_mapping = {'PERSON': 'PERSON',
                    'DATE': 'DATE_TIME',
                    'HOSPITAL': 'ORG',
                    'PATIENT': 'PERSON',
                    'MEDICALRECORD': 'AGE',
                    'AGE': 'AGE',
                    'COUNTRY': 'LOCATION',
                    'IDNUM': 'IDNUM',
                    'DOCTOR': 'PERSON',
                    'USERNAME': 'USERNAME',
                    'STREET': 'LOCATION',
                    'CITY': 'LOCATION',
                    'STATE': 'LOCATION',
                    'ZIP': 'LOCATION',
                    'PROFESSION': 'PROFESSION',
                    'PHONE': 'PHONE_NUMBER',
                    'ORGANIZATION': 'ORG',
                    'BIOID': 'BIOID',
                    'DEVICE': 'DEVICE',
                    'LOCATION-OTHER': 'LOCATION',
                    'PHONE_NUMBER': 'PHONE_NUMBER',
                    'EMAIL_ADDRESS': 'EMAIL_ADDRESS',
                    'HEALTHPLAN': 'HEALTHPLAN',
                    'URL': 'URL',
                    'O': 'O'}

new_list = ModelEvaluator.align_input_samples_to_presidio_analyzer(input_samples,
                                                                   entities_mapping)

#### D. Recalculate statistics on updated dataset

In [13]:
## recheck counter
count_per_entity_new = Counter([span.entity_type for span in flatten([input_sample.spans for input_sample in new_list])])
count_per_entity_new

Counter({'PERSON': 1235,
         'ORG': 404,
         'AGE': 507,
         'PROFESSION': 72,
         'IDNUM': 87,
         'USERNAME': 45,
         'PHONE_NUMBER': 48,
         'EMAIL_ADDRESS': 1})

#### E. Run the presidio-evaluator framework with Presidio's API as the 'model' at test

In [14]:
from presidio_evaluator import PresidioAPIEvaluator
presidio = PresidioAPIEvaluator(all_fields=False, entities_to_keep=list(count_per_entity_new.keys()),endpoint=MY_PRESIDIO_ENDPOINT)
evaluted_samples = presidio.evaluate_all(new_list)

#### F. Extract statistics
- Presicion, recall and F measure are calculated based on a PII/Not PII binary classification per token.
- Specific entity recall and precision are calculated on the specific PII entity level.

In [55]:
evaluation_result = presidio.calculate_score(evaluted_samples)

In [8]:
evaluation_result.print() #1 

                        Entity                     Precision                        Recall
                     DATE_TIME                        52.61%                        88.05%
                        PERSON                        94.48%                        72.49%
                           ORG                        50.58%                        58.99%
                      LOCATION                        72.66%                        38.75%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        70.68%
                 EMAIL_ADDRESS                       100.00%                       100.00%

In [13]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                     DATE_TIME                        52.61%                        88.05%
                        PERSON                        93.68%                        73.33%
                           ORG                        50.58%                        58.99%
                      LOCATION                        72.66%                        38.75%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        70.68%
                 EMAIL_ADDRESS                       100.00%                       100.00%

In [23]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                     DATE_TIME                        52.61%                        88.05%
                        PERSON                        86.81%                        75.30%
                           ORG                        50.58%                        58.99%
                      LOCATION                        72.82%                        38.56%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        70.68%
                 EMAIL_ADDRESS                       100.00%                       100.00%

In [16]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                     DATE_TIME                        52.61%                        88.05%
                        PERSON                        89.51%                        75.14%
                           ORG                        50.48%                        58.74%
                      LOCATION                        72.57%                        38.56%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        70.68%
                 EMAIL_ADDRESS                       100.00%                       100.00%

In [28]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                     DATE_TIME                        52.61%                        88.05%
                        PERSON                        79.71%                        78.24%
                           ORG                        50.58%                        58.74%
                      LOCATION                        72.28%                        38.01%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        70.68%
                 EMAIL_ADDRESS                       100.00%                       100.00%

In [32]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                     DATE_TIME                        52.61%                        88.05%
                        PERSON                        87.11%                        78.24%
                           ORG                        50.69%                        58.99%
                      LOCATION                        72.38%                        38.19%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        70.68%
                 EMAIL_ADDRESS                       100.00%                       100.00%

In [38]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                     DATE_TIME                        52.61%                        88.05%
                        PERSON                        87.24%                        78.24%
                           ORG                        50.69%                        58.99%
                      LOCATION                        72.47%                        38.38%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        70.68%
                 EMAIL_ADDRESS                       100.00%                       100.00%

In [45]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                     DATE_TIME                        52.65%                        88.05%
                        PERSON                        85.76%                        72.70%
                           ORG                        51.35%                        58.74%
                      LOCATION                        76.21%                        37.82%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        70.68%
                 EMAIL_ADDRESS                       100.00%                       100.00%

In [53]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                     DATE_TIME                        52.65%                        88.05%
                        PERSON                        85.38%                        79.58%
                           ORG                        51.35%                        58.74%
                      LOCATION                        76.21%                        37.82%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        70.68%
                 EMAIL_ADDRESS                       100.00%                       100.00%

In [57]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                     DATE_TIME                        52.65%                        88.05%
                        PERSON                        85.38%                        79.58%
                           ORG                        51.35%                        58.74%
                      LOCATION                        76.21%                        37.82%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        70.68%
                 EMAIL_ADDRESS                       100.00%                       100.00%

In [88]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                     DATE_TIME                        52.65%                        88.05%
                        PERSON                        83.80%                        82.18%
                           ORG                        51.68%                        58.62%
                      LOCATION                        77.36%                        37.82%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        70.68%
                 EMAIL_ADDRESS                       100.00%                       100.00%

In [98]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                     DATE_TIME                        52.65%                        88.05%
                        PERSON                        83.25%                        83.35%
                           ORG                        51.91%                        58.62%
                      LOCATION                        75.09%                        37.82%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        70.68%
                 EMAIL_ADDRESS                       100.00%                       100.00%

In [15]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                     DATE_TIME                        52.65%                        88.05%
                        PERSON                        82.53%                        83.98%
                           ORG                        51.74%                        58.62%
                      LOCATION                        78.24%                        37.82%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        70.68%
                 EMAIL_ADDRESS                       100.00%                       100.00%

In [30]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                     DATE_TIME                        52.65%                        88.05%
                        PERSON                        82.55%                        85.49%
                           ORG                        51.91%                        58.62%
                      LOCATION                        78.85%                        37.82%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        70.68%
                 EMAIL_ADDRESS                       100.00%                       100.00%

In [40]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                        PERSON                        65.05%                        87.88%
                           ORG                        52.88%                        56.53%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        66.92%
                 EMAIL_ADDRESS                       100.00%                       100.00%
                           PII                        64.31%                        59.40%
PII F measure: 0.6175761056863871


In [9]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                        PERSON                        65.25%                        85.58%
                           ORG                        52.57%                        56.77%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        66.92%
                 EMAIL_ADDRESS                       100.00%                       100.00%
                           PII                        64.30%                        58.19%
PII F measure: 0.6109048723897912


In [17]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                        PERSON                        65.25%                        85.58%
                           ORG                        52.57%                        56.77%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        66.92%
                 EMAIL_ADDRESS                       100.00%                       100.00%
                           PII                        64.30%                        58.19%
PII F measure: 0.6109048723897912


In [9]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                        PERSON                        65.03%                        89.81%
                           ORG                        53.00%                        56.53%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        66.92%
                 EMAIL_ADDRESS                       100.00%                       100.00%
                           PII                        64.35%                        60.44%
PII F measure: 0.6233618233618233


In [9]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                        PERSON                        65.03%                        89.81%
                           ORG                        53.00%                        56.53%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        66.92%
                 EMAIL_ADDRESS                       100.00%                       100.00%
                           PII                        64.35%                        60.44%
PII F measure: 0.6233618233618233


In [9]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                        PERSON                        51.05%                        89.81%
                           ORG                        17.53%                        34.36%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        66.92%
                 EMAIL_ADDRESS                       100.00%                       100.00%
                           PII                        44.66%                        58.01%
PII F measure: 0.5046621166971066


In [29]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                        PERSON                        60.59%                        91.40%
                           ORG                        52.94%                        56.53%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        66.92%
                 EMAIL_ADDRESS                       100.00%                       100.00%
                           PII                        60.88%                        61.28%
PII F measure: 0.6107929515418502


In [64]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                        PERSON                        60.55%                        91.53%
                           ORG                        53.19%                        56.53%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        66.92%
                 EMAIL_ADDRESS                       100.00%                       100.00%
                           PII                        60.86%                        61.30%
PII F measure: 0.6108114059231531


In [45]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                        PERSON                        60.57%                        91.99%
                           ORG                        53.19%                        56.53%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        66.92%
                 EMAIL_ADDRESS                       100.00%                       100.00%
                           PII                        60.87%                        61.55%
PII F measure: 0.6120879120879121


In [56]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                        PERSON                        59.67%                        93.75%
                           ORG                        51.81%                        52.83%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                         0.00%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                         0.00%
                  PHONE_NUMBER                       100.00%                        66.92%
                 EMAIL_ADDRESS                       100.00%                       100.00%
                           PII                        59.98%                        61.83%
PII F measure: 0.6089227421109902


#### G. Analyze wrong predictions

In [57]:
errors = evaluation_result.model_errors

In [ ]:
ModelEvaluator.most_common_fp_tokens(errors,n=5)

In [42]:
fps_df = ModelEvaluator.get_fps_dataframe(errors,entity='PERSON')
fps_df

,error_type,annotation,prediction,token,full_text,0
0,FP,O,PERSON,Medical,\n\n\nRecord date: 2092-11-16\n\n \n \n \n \n ...,None
1,FP,O,PERSON,Group,\n\n\nRecord date: 2092-11-16\n\n \n \n \n \n ...,None
2,FP,O,PERSON,\n,\n\n\nRecord date: 2092-11-16\n\n \n \n \n \n ...,None
3,FP,O,PERSON,\n,\n\n\nRecord date: 2092-11-16\n\n \n \n \n \n ...,None
4,FP,O,PERSON,MR,\n\n\nRecord date: 2092-11-16\n\n \n \n \n \n ...,None
...,...,...,...,...,...,...
1085,FP,O,PERSON,if,\n\n\nRecord date: 2088-10-18\n\n64 yo for f/u...,None
1086,FP,O,PERSON,her,\n\n\nRecord date: 2088-10-18\n\n64 yo for f/u...,None
1087,FP,O,PERSON,LENIS,\n\n\nRecord date: 2064-02-25\n\n\n\n\n\nINTER...,None
1088,FP,O,PERSON,MD,\n\n\nRecord date: 2077-07-03\n\n\n\n\n\nINTER...,None


In [58]:
fns_df = ModelEvaluator.get_fns_dataframe(errors,entity='ORG')
fns_df

In [19]:
input_samples[6001]

Full text: Other analysts were more bullish, even though the company is expected to shrink to slightly more than half its current size in sales.
Spans: []
Tokens: [Other, analysts, were, more, bullish, ,, even, though, the, company, is, expected, to, shrink, to, slightly, more, than, half, its, current, size, in, sales, .]
Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [3]:
for idx, inp in enumerate(input_samples):
    if "OCAMPO, NATHAN" in inp.full_text:
        break
idx

7

In [17]:
result

In [19]:
presidio.calculate_score([result]).print()

                        Entity                     Precision                        Recall
                        PERSON                        87.50%                        77.78%
                           ORG                          nan%                         0.00%
                           AGE                          nan%                         0.00%
                    PROFESSION                          nan%                          nan%
                         IDNUM                          nan%                         0.00%
                      USERNAME                          nan%                          nan%
                  PHONE_NUMBER                          nan%                         0.00%
                 EMAIL_ADDRESS                          nan%                          nan%
                           PII                        87.50%                        35.00%
PII F measure: 0.4999999999999999


In [4]:
input_samples[7]

Full text: 


Record date: 2093-09-13


HMS Cardiovascular Division
CVD R56
82 Ferry Road                                      OCAMPO, NATHAN-OLIVER
Foxborough, TX 58771                                           80531359
(564) 243-4471                                           09/13/93


Kenneth X. Sylvester, Jr., M.D.
Holden District Hospital
62 Ferry Road Cardiology
Foxborough, TX  58771


Dear Ken:

Mr. Ocampo returns to the office following his recent coronary artery bypass graft surgery.  As you know, Dr. Todd Banks took him to the operating room on August 17th, at which point, he underwent multivessel bypass surgery with placement of a left internal mammary artery to his LAD and reverse saphenous vein grafts to his PDA, first obtuse marginal, and second obtuse marginal.  He tolerated the procedure uneventfully.  His postoperative course was notable for intermittent atrial fibrillation for which he was eventually started on amiodarone therapy.  He also had some difficulty with uri